## 1. General

This section of for imports, getting an overview of all experiments, and loading the most suitable model.

In [3]:
import sys
import os
import warnings
import logging
from absl import logging as absl_logging

# ✅ Ensure Correct TensorFlow Configuration
os.environ['TFDS_DATA_DIR'] = r"/data/newc6477/VAE/Mean_Beat/5_percent_Physionet/"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

# ✅ Ensure Correct Working Directory
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print(f"📂 Project Root: {PROJECT_ROOT}")

# ✅ Ensure `src/` is in Python's Path
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

print(f"✅ Updated sys.path:\n{sys.path}")

# ✅ Suppress Warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
absl_logging.set_verbosity(absl_logging.ERROR)

# ✅ Import Modules
try:
    from src.utils.helper import Helper
    from src.evaluate.visualizations import Visualizations
    print("✅ Successfully imported `Helper` and `Visualizations`")
except ModuleNotFoundError as e:
    print(f"❌ Import Error: {e}")
    print("🔍 Check if `src/` has `__init__.py` and is in `sys.path`.")

# ✅ Import Other Required Libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import glob
from neurokit2.signal import signal_smooth

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

import ipywidgets as widgets
from matplotlib import pyplot as plt
import seaborn as sns



📂 Project Root: /users/newc6477/VAE/12_Lead_VECG
✅ Updated sys.path:
['/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python310.zip', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10/lib-dynload', '', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10/site-packages', '/users/newc6477/VAE/12_Lead_VECG/analysis', '/users/newc6477/VAE/12_Lead_VECG/analysis/src', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10/site-packages/setuptools/_vendor', '/users/newc6477/VAE/12_Lead_VECG', '/users/newc6477/VAE/12_Lead_VECG/src']
✅ Successfully imported `Helper` and `Visualizations`


In [4]:
# The resolution for saving images
DPI = 300

# The source path of the experiments and models
PATH = r"/users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/"

# Some operations take some time in computation.
# Therefore, the stored intermediate results can be used to skip the respective computation.
USE_PRECOMPUTED = True

# 2. Anomaly Detection

## 🔹 Funcs used to train and test

In [5]:
import json
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
import pandas as pd

def evaluate_models(df_physionet_train, df_physionet_test, hyperparams_list):
    """
    Evaluate a list of models using KNN classification and return a summary table.
    
    Args:
        df_physionet_train (List[pd.DataFrame]): List of train embedding DataFrames (1 per model).
        df_physionet_test (List[pd.DataFrame]): List of test embedding DataFrames (1 per model).
        hyperparams_list (List[dict]): Corresponding list of hyperparameter dictionaries.
    
    Returns:
        pd.DataFrame: Summary table with accuracy, F1 scores, hyperparameters.
    """
    results = []

    for i in range(len(df_physionet_train)):
        print(f"\n🔍 Evaluating model {i+1}/{len(df_physionet_train)}...")

        # Extract hyperparameters and latent dimension
        hparams = hyperparams_list[i]
        latent_dim = hparams.get("latent_dimension", 8)

        # Prepare feature matrices and labels
        X_train = df_physionet_train[i].iloc[:, :latent_dim].values  
        X_test = df_physionet_test[i].iloc[:, :latent_dim].values  
        y_train = np.array(df_physionet_train[i]['diagnostic'].tolist(), dtype=int)  
        y_test = np.array(df_physionet_test[i]['diagnostic'].tolist(), dtype=int)  

        # KNN classification with hyperparameter tuning
        knn = KNeighborsClassifier()
        multi_knn = MultiOutputClassifier(knn, n_jobs=-1)

        param_grid = {"estimator__n_neighbors": [3, 5, 7, 9, 11]}
        grid = GridSearchCV(multi_knn, param_grid, scoring="accuracy", cv=3, n_jobs=-1)
        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)

        # Evaluation metrics
        overall_acc = accuracy_score(y_test, y_pred)
        f1_micro = f1_score(y_test, y_pred, average="micro")
        f1_macro = f1_score(y_test, y_pred, average="macro")
        accuracy_per_label = (y_pred == y_test).mean(axis=0)

        # Print evaluation
        print("✅ Overall Accuracy:", overall_acc)
        print("✅ F1 Score (Macro):", f1_macro)
        print("✅ F1 Score (Micro):", f1_micro)
        print("✅ Accuracy per label:", accuracy_per_label)

        # Classification report
        print("✅ Classification Report:\n")
        print(classification_report(
            y_test, y_pred,
            target_names=[str(i) for i in range(y_test.shape[1])]
        ))

        # Collect results
        results.append({
            "model_index": i,
            "best_k": grid.best_params_["estimator__n_neighbors"],
            "accuracy": overall_acc,
            "f1_micro": f1_micro,
            "f1_macro": f1_macro,
            **hparams
        })

    summary_df = pd.DataFrame(results)
    return summary_df


def extract_hyperparams_from_json(json_path):
    with open(json_path, 'r') as f:
        params = json.load(f)

    # Extract key hyperparameters
    hyperparams = {
        "alpha": params["coefficients"]["alpha"],
        "beta": params["coefficients"]["beta"],
        "gamma": params["coefficients"]["gamma"],
        "latent_dimension": params["latent_dimension"],
        "learning_rate": params["learning_rate"],
        "epochs": params["epochs"]
    }

    return hyperparams

from IPython.display import display
def summarize_by_latent_dimension(results_df):
    unique_dims = results_df["latent_dimension"].unique()
    summary_tables = {}

    for dim in sorted(unique_dims):
        filtered = results_df[results_df["latent_dimension"] == dim]
        sorted_table = filtered.sort_values(by="f1_macro", ascending=False)
        summary_tables[dim] = sorted_table

        print(f"\n📏 Latent Dimension: {dim}")
        display(sorted_table)

    return summary_tables


## 🔹 Lead I Evaluation

In [6]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'I')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/I/2025-04-04_12-31-56/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/I/2025-04-04_12-33-05/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/I/2025-04-04_12-33-58/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/I/2025-04-04_12-34-48/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/I/2025-04-04_12-35-39/model_best.keras
Loaded 5 models successfully.


In [7]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}

In [8]:
df_physionet_train, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='I')
df_physionet_test, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='I')

#df_physionet_train = df_physionet_train[0]
#df_physionet_test = df_physionet_test[0]
#print(type(df_physionet_train))



📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: I
      1/Unknown - 0s 290ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 14ms/step
      1/Unknown - 0s 134ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 21ms/step
      1/Unknown - 0s 129ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step
      1/Unknown - 0s 131ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 17ms/step
      1/Unknown - 0s 123ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 25ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: I
✅ Generator exhausted normally.
1/1 [==============================] - 0s 53ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 38ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 4

In [9]:

results_df = evaluate_models(df_physionet_train,df_physionet_test, hyperparams_list)


🔍 Evaluating model 1/5...
✅ Overall Accuracy: 0.25170068027210885
✅ F1 Score (Macro): 0.11937647979425689
✅ F1 Score (Micro): 0.3400765445598688
✅ Accuracy per label: [0.94331066 0.89002268 0.99546485 0.99659864 0.98752834 0.96485261
 0.96598639 0.98185941 0.90816327 0.97732426 0.99433107 0.98412698
 0.98412698 0.98412698 0.67687075 0.97278912 0.98639456 0.99433107
 0.99092971 0.98072562 0.97959184 0.95351474 0.8276644  0.90249433
 0.85600907 0.95238095]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.25      0.02      0.04        48
           1       0.31      0.15      0.20        82
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         3
           4       1.00      0.42      0.59        19
           5       0.77      0.26      0.39        38
           6       0.00      0.00      0.00        30
           7       0.00      0.00      0.00        16
           8       0.40    

In [10]:
display(results_df.sort_values(by="f1_macro", ascending=False))

# Set your lead name (e.g., "I", "II")
lead_name = "I"  # change as needed

best_index = results_df.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,5,0.277778,0.378519,0.142559,2.01,0.3,0.2,24,0.002,150
1,1,5,0.264172,0.379024,0.139112,2.01,0.3,0.2,18,0.002,150
2,2,11,0.278912,0.382047,0.137835,2.01,0.3,0.2,20,0.002,150
0,0,7,0.251701,0.340077,0.119376,2.01,0.3,0.2,16,0.002,150
3,3,7,0.261905,0.340733,0.104449,2.01,0.3,0.2,22,0.002,150


## 🔹 Lead II Evaluation


In [11]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'II')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/II/2025-04-04_12-36-27/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/II/2025-04-04_12-37-18/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/II/2025-04-04_12-38-04/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/II/2025-04-04_12-38-56/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/II/2025-04-04_12-39-43/model_best.keras
Loaded 5 models successfully.


In [12]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_II, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='II')
df_physionet_test_II, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='II')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: II
      1/Unknown - 0s 135ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 17ms/step
      1/Unknown - 0s 119ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 23ms/step
      1/Unknown - 0s 116ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 16ms/step
      1/Unknown - 0s 118ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step
      1/Unknown - 0s 118ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: II
✅ Generator exhausted normally.
1/1 [==============================] - 0s 42ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 52ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s

In [13]:
results_df_II = evaluate_models(df_physionet_train_II,df_physionet_test_II, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.264172335600907
✅ F1 Score (Macro): 0.1300570012460168
✅ F1 Score (Micro): 0.37201907790143085
✅ Accuracy per label: [0.93650794 0.90929705 0.99546485 0.99659864 0.97732426 0.95464853
 0.96712018 0.98185941 0.91723356 0.97959184 0.99433107 0.98412698
 0.98412698 0.98412698 0.68480726 0.9739229  0.98639456 0.99433107
 0.99092971 0.98072562 0.9829932  0.95238095 0.84126984 0.89115646
 0.86281179 0.95238095]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.32      0.15      0.20        48
           1       0.52      0.29      0.38        82
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        19
           5       0.25      0.03      0.05        38
           6       0.67      0.07      0.12        30
           7       0.00      0.00      0.00        16
           8       0.62      0.23      0.33        80
  

In [14]:
display(results_df_II.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "II"  # change as needed

best_index = results_df_II.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,7,0.264172,0.372019,0.130057,2.01,0.3,0.2,16,0.002,150
1,1,9,0.285714,0.378465,0.119423,2.01,0.3,0.2,18,0.002,150
4,4,5,0.234694,0.342371,0.111237,2.01,0.3,0.2,24,0.002,150
3,3,11,0.277778,0.365237,0.106918,2.01,0.3,0.2,22,0.002,150
2,2,9,0.286848,0.380435,0.106055,2.01,0.3,0.2,20,0.002,150


## 🔹 Lead III Evaluation


In [15]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'III')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/III/2025-04-04_12-40-35/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/III/2025-04-04_12-41-21/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/III/2025-04-04_12-42-00/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/III/2025-04-04_12-42-40/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/III/2025-04-04_12-43-21/model_best.keras
Loaded 5 models successfully.


In [16]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_III, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='III')
df_physionet_test_III, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='III')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: III
      1/Unknown - 0s 120ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 18ms/step
      1/Unknown - 0s 120ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 17ms/step
      1/Unknown - 0s 116ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 18ms/step
      1/Unknown - 0s 117ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step
      1/Unknown - 0s 115ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 17ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: III
✅ Generator exhausted normally.
1/1 [==============================] - 0s 55ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 48ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 

In [17]:
results_df_III = evaluate_models(df_physionet_train_III,df_physionet_test_III, hyperparams_list)


🔍 Evaluating model 1/5...
✅ Overall Accuracy: 0.1984126984126984
✅ F1 Score (Macro): 0.0931446388886587
✅ F1 Score (Micro): 0.2927374301675978
✅ Accuracy per label: [0.9399093  0.8968254  0.99546485 0.99659864 0.97959184 0.95804989
 0.96598639 0.98185941 0.90249433 0.97505669 0.99433107 0.98412698
 0.98412698 0.98412698 0.67687075 0.97278912 0.98639456 0.99433107
 0.99092971 0.98072562 0.9829932  0.95124717 0.79705215 0.88321995
 0.85941043 0.95011338]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        48
           1       0.36      0.15      0.21        82
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         3
           4       1.00      0.05      0.10        19
           5       0.57      0.11      0.18        38
           6       0.00      0.00      0.00        30
           7       0.00      0.00      0.00        16
           8       0.42      

In [18]:
display(results_df_III.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "III"  # change as needed

best_index = results_df_III.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,5,0.198413,0.292737,0.093145,2.01,0.3,0.2,16,0.002,150
3,3,5,0.174603,0.258137,0.088414,2.01,0.3,0.2,22,0.002,150
4,4,5,0.191610,0.282022,0.085178,2.01,0.3,0.2,24,0.002,150
2,2,7,0.201814,0.301479,0.083969,2.01,0.3,0.2,20,0.002,150
1,1,7,0.178005,0.257576,0.077493,2.01,0.3,0.2,18,0.002,150


## 🔹 Lead aVR Evaluation


In [19]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'aVR')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVR/2025-04-04_12-44-00/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVR/2025-04-04_12-44-40/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVR/2025-04-04_12-45-20/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVR/2025-04-04_12-45-58/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVR/2025-04-04_12-46-38/model_best.keras
Loaded 5 models successfully.


In [20]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_aVR, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='aVR')
df_physionet_test_aVR, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='aVR')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: aVR
      1/Unknown - 0s 134ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step
      1/Unknown - 0s 133ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step
      1/Unknown - 0s 120ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 18ms/step
      1/Unknown - 0s 119ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 17ms/step
      1/Unknown - 0s 125ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 23ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: aVR
✅ Generator exhausted normally.
1/1 [==============================] - 0s 43ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 44ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 

In [21]:
results_df_aVR = evaluate_models(df_physionet_train_aVR,df_physionet_test_aVR, hyperparams_list)


🔍 Evaluating model 1/5...
✅ Overall Accuracy: 0.264172335600907
✅ F1 Score (Macro): 0.0834748802277734
✅ F1 Score (Micro): 0.32917139614074914
✅ Accuracy per label: [0.94444444 0.89795918 0.99546485 0.99659864 0.97845805 0.95464853
 0.96598639 0.98185941 0.91043084 0.97732426 0.99433107 0.98412698
 0.98412698 0.98412698 0.69614512 0.97278912 0.98866213 0.99433107
 0.99092971 0.98072562 0.9829932  0.95351474 0.84126984 0.88435374
 0.87188209 0.95238095]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.33      0.02      0.04        48
           1       0.21      0.04      0.06        82
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        19
           5       0.38      0.08      0.13        38
           6       0.50      0.03      0.06        30
           7       0.00      0.00      0.00        16
           8       1.00      

In [22]:
display(results_df_aVR.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "aVR"  # change as needed

best_index = results_df_aVR.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,7,0.263039,0.343490,0.098110,2.01,0.3,0.2,18,0.002,150
4,4,9,0.258503,0.333710,0.097844,2.01,0.3,0.2,24,0.002,150
3,3,9,0.261905,0.342566,0.087173,2.01,0.3,0.2,22,0.002,150
0,0,11,0.264172,0.329171,0.083475,2.01,0.3,0.2,16,0.002,150
2,2,11,0.265306,0.331624,0.070247,2.01,0.3,0.2,20,0.002,150


## 🔹 Lead aVL Evaluation


In [23]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'aVL')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVL/2025-04-04_12-49-46/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVL/2025-04-04_12-50-38/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVL/2025-04-04_12-51-19/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVL/2025-04-04_12-52-02/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVL/2025-04-04_12-52-43/model_best.keras
Loaded 5 models successfully.


In [24]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_aVL, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='aVL')
df_physionet_test_aVL, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='aVL')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: aVL
      1/Unknown - 0s 138ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 23ms/step
      1/Unknown - 0s 211ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 26ms/step
      1/Unknown - 0s 139ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step
      1/Unknown - 0s 120ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 23ms/step
      1/Unknown - 0s 127ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: aVL
✅ Generator exhausted normally.
1/1 [==============================] - 0s 45ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 49ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 

In [25]:
results_df_aVL = evaluate_models(df_physionet_train_aVL,df_physionet_test_aVL, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.17913832199546487
✅ F1 Score (Macro): 0.08175331428485605
✅ F1 Score (Micro): 0.2588495575221239
✅ Accuracy per label: [0.93650794 0.89569161 0.99546485 0.99659864 0.97959184 0.95918367
 0.96371882 0.98072562 0.88888889 0.97505669 0.99433107 0.98412698
 0.98412698 0.98412698 0.64512472 0.97165533 0.98185941 0.99433107
 0.99092971 0.98072562 0.97959184 0.95238095 0.78571429 0.87414966
 0.85487528 0.95124717]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.10      0.02      0.03        48
           1       0.29      0.09      0.13        82
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         3
           4       0.56      0.26      0.36        19
           5       0.57      0.21      0.31        38
           6       0.00      0.00      0.00        30
           7       0.00      0.00      0.00        16
           8       0.18      0.06      0.09        80


In [26]:
display(results_df_aVL.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "aVF"  # change as needed

best_index = results_df_aVL.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
3,3,5,0.182540,0.271018,0.092293,2.01,0.3,0.2,22,0.002,150
2,2,5,0.185941,0.266520,0.083111,2.01,0.3,0.2,20,0.002,150
0,0,5,0.179138,0.258850,0.081753,2.01,0.3,0.2,16,0.002,150
4,4,3,0.172336,0.266943,0.079136,2.01,0.3,0.2,24,0.002,150
1,1,9,0.178005,0.227384,0.047696,2.01,0.3,0.2,18,0.002,150


## aVF

In [27]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'aVF')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVF/2025-04-04_12-53-25/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVF/2025-04-04_12-54-06/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVF/2025-04-04_12-54-47/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVF/2025-04-04_12-55-29/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/aVF/2025-04-04_12-56-10/model_best.keras
Loaded 5 models successfully.


In [28]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_aVF, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='aVF')
df_physionet_test_aVF, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='aVF')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: aVF
      1/Unknown - 0s 123ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 28ms/step
      1/Unknown - 0s 144ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 23ms/step
      1/Unknown - 0s 119ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 21ms/step
      1/Unknown - 0s 121ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 23ms/step
      1/Unknown - 0s 117ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 24ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: aVF
✅ Generator exhausted normally.
1/1 [==============================] - 0s 45ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 59ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 

In [29]:
results_df_aVF = evaluate_models(df_physionet_train_aVF,df_physionet_test_aVF, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.2290249433106576
✅ F1 Score (Macro): 0.08673726103100673
✅ F1 Score (Micro): 0.30419977298524403
✅ Accuracy per label: [0.94104308 0.91043084 0.99546485 0.99659864 0.97732426 0.95578231
 0.96598639 0.98072562 0.90816327 0.97845805 0.99433107 0.9829932
 0.98412698 0.98412698 0.6723356  0.97278912 0.98526077 0.99433107
 0.99092971 0.98072562 0.9829932  0.95351474 0.80498866 0.89569161
 0.86848073 0.95238095]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.36      0.10      0.16        48
           1       0.56      0.18      0.28        82
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        19
           5       0.00      0.00      0.00        38
           6       0.00      0.00      0.00        30
           7       0.00      0.00      0.00        16
           8       0.49      0.25      0.33        80
 

In [30]:
display(results_df_aVF.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "aVF"  # change as needed

best_index = results_df_aVF.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,5,0.226757,0.319489,0.113374,2.01,0.3,0.2,18,0.002,150
4,4,3,0.225624,0.318709,0.111449,2.01,0.3,0.2,24,0.002,150
2,2,3,0.208617,0.306680,0.100734,2.01,0.3,0.2,20,0.002,150
0,0,7,0.229025,0.304200,0.086737,2.01,0.3,0.2,16,0.002,150
3,3,5,0.231293,0.314130,0.082957,2.01,0.3,0.2,22,0.002,150


## 🔹 Lead V1 Evaluation

In [31]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V1')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V1/2025-04-04_12-56-52/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V1/2025-04-04_12-57-33/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V1/2025-04-04_12-58-13/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V1/2025-04-04_12-58-53/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V1/2025-04-04_12-59-33/model_best.keras
Loaded 5 models successfully.


In [32]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V1, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V1')
df_physionet_test_V1, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V1')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V1
      3/Unknown - 0s 34ms/step ✅ Generator exhausted normally.
4/4 [==============================] - 0s 32ms/step
      1/Unknown - 0s 118ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 21ms/step
      1/Unknown - 0s 116ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 26ms/step
      1/Unknown - 0s 162ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 25ms/step
      1/Unknown - 0s 184ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 19ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V1
✅ Generator exhausted normally.
1/1 [==============================] - 0s 39ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 50ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s

In [33]:
results_df_V1 = evaluate_models(df_physionet_train_V1,df_physionet_test_V1, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.2891156462585034
✅ F1 Score (Macro): 0.10853027535800647
✅ F1 Score (Micro): 0.38461538461538464
✅ Accuracy per label: [0.94331066 0.90022676 0.99546485 0.99659864 0.97959184 0.97278912
 0.96598639 0.9829932  0.90929705 0.97732426 0.99433107 0.98412698
 0.98185941 0.98412698 0.71428571 0.97278912 0.98526077 0.99433107
 0.99092971 0.98072562 0.9829932  0.95351474 0.85714286 0.90816327
 0.86961451 0.95238095]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.33      0.04      0.07        48
           1       0.31      0.06      0.10        82
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         3
           4       1.00      0.05      0.10        19
           5       0.67      0.74      0.70        38
           6       0.00      0.00      0.00        30
           7       1.00      0.06      0.12        16
           8       0.00      0.00      0.00        80


In [34]:
display(results_df_V1.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V1"  # change as needed

best_index = results_df_V1.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
2,2,11,0.291383,0.389468,0.112077,2.01,0.3,0.2,20,0.002,150
1,1,9,0.268707,0.374725,0.109446,2.01,0.3,0.2,18,0.002,150
0,0,11,0.289116,0.384615,0.108530,2.01,0.3,0.2,16,0.002,150
3,3,9,0.283447,0.366942,0.107000,2.01,0.3,0.2,22,0.002,150
4,4,9,0.273243,0.359452,0.100792,2.01,0.3,0.2,24,0.002,150


## 🔹 Lead V2 Evaluation


In [35]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V2')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V2/2025-04-04_13-00-13/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V2/2025-04-04_13-00-54/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V2/2025-04-04_13-01-35/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V2/2025-04-04_13-02-13/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V2/2025-04-04_13-02-52/model_best.keras
Loaded 5 models successfully.


In [36]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V2, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V2')
df_physionet_test_V2, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V2')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V2
      1/Unknown - 0s 125ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 16ms/step
      1/Unknown - 0s 122ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 21ms/step
      1/Unknown - 0s 121ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 17ms/step
      1/Unknown - 0s 182ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 21ms/step
      1/Unknown - 0s 120ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 22ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V2
✅ Generator exhausted normally.
1/1 [==============================] - 0s 43ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 54ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s

In [37]:
results_df_V2 = evaluate_models(df_physionet_train_V2,df_physionet_test_V2, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.2653061224489796
✅ F1 Score (Macro): 0.09916303544452826
✅ F1 Score (Micro): 0.3643410852713178
✅ Accuracy per label: [0.94557823 0.89795918 0.99546485 0.99659864 0.97732426 0.96598639
 0.96598639 0.98185941 0.90929705 0.97732426 0.99433107 0.98412698
 0.98412698 0.98412698 0.68480726 0.97278912 0.98526077 0.99433107
 0.99092971 0.98072562 0.9829932  0.95351474 0.87188209 0.89795918
 0.8707483  0.95238095]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        48
           1       0.28      0.06      0.10        82
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         3
           4       0.40      0.11      0.17        19
           5       0.67      0.42      0.52        38
           6       0.00      0.00      0.00        30
           7       0.00      0.00      0.00        16
           8       0.00      0.00      0.00        80
 

In [38]:
display(results_df_V2.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V2"  # change as needed

best_index = results_df_V2.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,11,0.265306,0.364341,0.099163,2.01,0.3,0.2,16,0.002,150
3,3,9,0.278912,0.368450,0.098363,2.01,0.3,0.2,22,0.002,150
4,4,9,0.268707,0.357616,0.096816,2.01,0.3,0.2,24,0.002,150
1,1,9,0.290249,0.380220,0.095417,2.01,0.3,0.2,18,0.002,150
2,2,11,0.281179,0.367460,0.088462,2.01,0.3,0.2,20,0.002,150


## 🔹 Lead V3 Evaluation


In [56]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V3')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V3/2025-04-04_13-03-32/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V3/2025-04-04_13-04-12/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V3/2025-04-04_13-04-50/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V3/2025-04-04_13-05-29/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V3/2025-04-04_13-06-08/model_best.keras
Loaded 5 models successfully.


In [57]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V3, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V3')
df_physionet_test_V3, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V3')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']


🔍 Processing lead: V3
      3/Unknown - 2s 85ms/step✅ Generator exhausted normall
4/4 [==============================] - 2s 156ms/step
      3/Unknown - 3s 116ms/step✅ Generator exhausted normally.
4/4 [==============================] - 3s 134ms/step
      1/Unknown - 2s 2s/step✅ Generator exhausted normally.
4/4 [==============================] - 3s 201ms/step
      2/Unknown - 2s 124ms/step✅ Generator exhausted normally.
4/4 [==============================] - 3s 151ms/step
      3/Unknown - 2s 30ms/step✅ Generator exhausted normally.
4/4 [==============================] - 3s 107ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V3
✅ Generator exhausted normally.
1/1 [==============================] - 0s 299ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 367ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 63ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 

In [58]:
results_df_V3 = evaluate_models(df_physionet_train_V3,df_physionet_test_V3, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.26643990929705214
✅ F1 Score (Macro): 0.0824610593249172
✅ F1 Score (Micro): 0.3522791221159257
✅ Accuracy per label: [0.94557823 0.89795918 0.99546485 0.99659864 0.98185941 0.95351474
 0.96598639 0.98185941 0.90929705 0.97732426 0.99433107 0.98412698
 0.98412698 0.98412698 0.70181406 0.97278912 0.98412698 0.99433107
 0.99092971 0.98072562 0.9829932  0.95351474 0.87755102 0.87981859
 0.87188209 0.95238095]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        48
           1       0.21      0.04      0.06        82
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         3
           4       0.71      0.26      0.38        19
           5       0.29      0.05      0.09        38
           6       0.00      0.00      0.00        30
           7       0.00      0.00      0.00        16
           8       0.00      0.00      0.00        80
 

In [59]:
display(results_df_V3.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V3"  # change as needed

best_index = results_df_V3.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
2,2,11,0.299320,0.380688,0.095296,2.01,0.3,0.2,20,0.002,150
3,3,11,0.261905,0.353665,0.091925,2.01,0.3,0.2,22,0.002,150
1,1,11,0.274376,0.364537,0.090407,2.01,0.3,0.2,18,0.002,150
4,4,11,0.261905,0.355828,0.082932,2.01,0.3,0.2,24,0.002,150
0,0,11,0.266440,0.352279,0.082461,2.01,0.3,0.2,16,0.002,150


## 🔹 Lead V4 Evaluation


In [43]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V4')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V4/2025-04-04_13-06-49/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V4/2025-04-04_13-07-30/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V4/2025-04-04_13-08-10/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V4/2025-04-04_13-08-51/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V4/2025-04-04_13-09-31/model_best.keras
Loaded 5 models successfully.


In [44]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V4, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V4')
df_physionet_test_V4, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V4')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V4
      1/Unknown - 0s 138ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step
      1/Unknown - 0s 122ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 21ms/step
      1/Unknown - 0s 121ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step
      1/Unknown - 0s 117ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 17ms/step
      1/Unknown - 0s 155ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 19ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V4
✅ Generator exhausted normally.
1/1 [==============================] - 0s 42ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 41ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s

In [45]:
results_df_V4 = evaluate_models(df_physionet_train_V4,df_physionet_test_V4, hyperparams_list)


🔍 Evaluating model 1/5...
✅ Overall Accuracy: 0.2732426303854875
✅ F1 Score (Macro): 0.08239341538331238
✅ F1 Score (Micro): 0.3619806488332385
✅ Accuracy per label: [0.94557823 0.89795918 0.99546485 0.99659864 0.98412698 0.95578231
 0.96598639 0.98185941 0.90929705 0.97732426 0.99433107 0.98412698
 0.98412698 0.98412698 0.71655329 0.97278912 0.98526077 0.99433107
 0.99092971 0.98072562 0.9829932  0.95351474 0.88095238 0.89115646
 0.8707483  0.95238095]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.50      0.02      0.04        48
           1       0.00      0.00      0.00        82
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         3
           4       0.86      0.32      0.46        19
           5       0.00      0.00      0.00        38
           6       0.00      0.00      0.00        30
           7       0.00      0.00      0.00        16
           8       0.00     

In [46]:
display(results_df_V4.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V4"  # change as needed

best_index = results_df_V4.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,11,0.273243,0.361981,0.082393,2.01,0.3,0.2,16,0.002,150
2,2,9,0.276644,0.366778,0.081882,2.01,0.3,0.2,20,0.002,150
4,4,11,0.257370,0.342825,0.080721,2.01,0.3,0.2,24,0.002,150
1,1,7,0.267574,0.344524,0.074723,2.01,0.3,0.2,18,0.002,150
3,3,9,0.264172,0.343820,0.073842,2.01,0.3,0.2,22,0.002,150


## 🔹 Lead V5 Evaluation


In [47]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V5')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V5/2025-04-04_13-10-11/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V5/2025-04-04_13-10-52/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V5/2025-04-04_13-11-31/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V5/2025-04-04_13-12-11/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V5/2025-04-04_13-12-50/model_best.keras
Loaded 5 models successfully.


In [48]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V5, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V5')
df_physionet_test_V5, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V5')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V5
      1/Unknown - 0s 120ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 23ms/step
      1/Unknown - 0s 118ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 18ms/step
      1/Unknown - 0s 118ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 21ms/step
      1/Unknown - 0s 130ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step
      1/Unknown - 0s 121ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V5
✅ Generator exhausted normally.
1/1 [==============================] - 0s 39ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 47ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s

In [49]:
results_df_V5 = evaluate_models(df_physionet_train_V5,df_physionet_test_V5, hyperparams_list)


🔍 Evaluating model 1/5...
✅ Overall Accuracy: 0.2800453514739229
✅ F1 Score (Macro): 0.07451653335665265
✅ F1 Score (Micro): 0.35482030804335424
✅ Accuracy per label: [0.94444444 0.9047619  0.99546485 0.99659864 0.97959184 0.9569161
 0.96598639 0.98185941 0.90816327 0.97732426 0.99433107 0.98412698
 0.98412698 0.98412698 0.72108844 0.97278912 0.98526077 0.99433107
 0.99092971 0.98072562 0.9829932  0.95351474 0.8537415  0.90136054
 0.8707483  0.95238095]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        48
           1       0.45      0.11      0.18        82
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         3
           4       1.00      0.05      0.10        19
           5       0.00      0.00      0.00        38
           6       0.00      0.00      0.00        30
           7       0.00      0.00      0.00        16
           8       0.00     

In [50]:
display(results_df_V5.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V5"  # change as needed

best_index = results_df_V5.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
3,3,9,0.282313,0.357063,0.083101,2.01,0.3,0.2,22,0.002,150
1,1,9,0.265306,0.342603,0.078607,2.01,0.3,0.2,18,0.002,150
0,0,11,0.280045,0.354820,0.074517,2.01,0.3,0.2,16,0.002,150
4,4,11,0.268707,0.339884,0.070852,2.01,0.3,0.2,24,0.002,150
2,2,11,0.286848,0.354072,0.067312,2.01,0.3,0.2,20,0.002,150


## 🔹 Lead V6 Evaluation


In [51]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V6')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V6/2025-04-04_13-13-31/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V6/2025-04-04_13-14-12/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V6/2025-04-04_13-14-57/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V6/2025-04-04_13-15-38/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_5_percent/test_is_split1/V6/2025-04-04_13-16-18/model_best.keras
Loaded 5 models successfully.


In [52]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V6, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V6')
df_physionet_test_V6, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V6')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V6
      1/Unknown - 0s 122ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 19ms/step
      1/Unknown - 0s 123ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step
      1/Unknown - 0s 117ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 20ms/step
      1/Unknown - 0s 117ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 25ms/step
      1/Unknown - 0s 121ms/step✅ Generator exhausted normally.
4/4 [==============================] - 0s 23ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V6
✅ Generator exhausted normally.
1/1 [==============================] - 0s 47ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s 43ms/step
✅ Generator exhausted normally.
1/1 [==============================] - 0s

In [53]:
results_df_V6 = evaluate_models(df_physionet_train_V6,df_physionet_test_V6, hyperparams_list)


🔍 Evaluating model 1/5...
✅ Overall Accuracy: 0.25396825396825395
✅ F1 Score (Macro): 0.08425178839573394
✅ F1 Score (Micro): 0.31776765375854216
✅ Accuracy per label: [0.94557823 0.90929705 0.99546485 0.99659864 0.98185941 0.95804989
 0.96598639 0.98185941 0.90362812 0.97845805 0.99433107 0.98412698
 0.98412698 0.98412698 0.68367347 0.97278912 0.98526077 0.99433107
 0.99092971 0.98072562 0.9829932  0.95238095 0.83673469 0.87755102
 0.86848073 0.95238095]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.50      0.02      0.04        48
           1       0.56      0.12      0.20        82
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         3
           4       0.67      0.32      0.43        19
           5       1.00      0.03      0.05        38
           6       0.00      0.00      0.00        30
           7       0.00      0.00      0.00        16
           8       0.14   

In [54]:
display(results_df_V6.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V6"  # change as needed

best_index = results_df_V6.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
3,3,7,0.274376,0.348634,0.107059,2.01,0.3,0.2,22,0.002,150
1,1,7,0.248299,0.328308,0.097400,2.01,0.3,0.2,18,0.002,150
4,4,7,0.239229,0.308546,0.094286,2.01,0.3,0.2,24,0.002,150
0,0,9,0.253968,0.317768,0.084252,2.01,0.3,0.2,16,0.002,150
2,2,9,0.216553,0.288540,0.083584,2.01,0.3,0.2,20,0.002,150


## 🔹 Special Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = '/users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II'  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-14-42/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-22-20/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-30-02/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-37-56/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-45-54/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_14-28-26/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_14-36-43/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_14-44-48/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/l

In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='II')
df_physionet_test, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='II')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']


🔍 Processing lead: II
     43/Unknown - 2s 50ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 50ms/step
     42/Unknown - 2s 46ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 45ms/step
     43/Unknown - 2s 48ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 48ms/step
     43/Unknown - 2s 46ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 46ms/step
     42/Unknown - 2s 47ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 47ms/step
     42/Unknown - 2s 46ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 46ms/step
     43/Unknown - 2s 48ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 47ms/step
     43/Unknown - 2s 48ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 48ms/step
     43/Unknown - 2s 46ms/step✅ Generator exhauste